# BrewER Experiments
Este notebook replica as funcionalidades do experiments.ipynb para o algoritmo BrewER

In [1]:
import pandas as pd
import pickle as pkl
from collections import deque
import time
import itertools as it
import numpy as np
import math
import pprint

import re

from brewer import Task, brewer, blocking, matching, fusion, parser

import variables as var

from itertools import combinations

In [2]:
# Change to raw dataset path
data = pd.read_csv("datasets/alaska_cameras_small/dataset.csv")

# Change to matches (dataframe with l_id and r_id pairs)
matches = pd.read_csv("datasets/alaska_cameras_small/matches.csv")

# Change to a pickled candidate pairs file (list of pairs)
with open(
    "datasets/alaska_cameras_small/blocking_functions/candidates_sparker.pkl",
    "rb",
) as f:
    candidates = pkl.load(f)

In [3]:
# If your matches aren't using l_id and r_id, or you have a different matching function change here
def matcher(l, r):
    return (
        ((matches["l_id"] == l) & (matches["r_id"] == r))
        | ((matches["l_id"] == r) & (matches["r_id"] == l))
    ).any()

In [4]:
# Change based on the aggregation functions you want to use for BrewER
def resolve_batch(records):
    entity = {}
    entity["_id"] = records['_id'].mode()[0]
    entity["description"] = records['description'].mode()[0]
    entity["brand"] = records['brand'].mode()[0]
    entity["price"] = records["price"].min()
    entity["mp"] = records["mp"].mean()
    return entity

In [5]:
def match_batch(candidate_pairs):
    seen = set()
    matches_set = set()
    not_matches_set = set()
    result = []
    comparisons = 0
    for idx, row in data.iterrows():
        # Change if your id column is different
        root = row["_id"]
        if root in seen:
            continue

        pairs = [pair for pair in candidate_pairs if root in pair]
        if not pairs:
            continue

        block = set().union(*pairs)

        entity_cluster = set([root])
        to_analyze = deque([root])

        while to_analyze:
            id = to_analyze.popleft()
            for candidate in block:
                if candidate in entity_cluster:
                    continue
                if id == candidate:
                    entity_cluster.add(candidate)
                    continue
                if (id, candidate) in matches_set:
                    entity_cluster.add(candidate)
                    to_analyze.append(candidate)
                    continue
                if (id, candidate) in not_matches_set:
                    continue
                comparisons += 1
                if matcher(id, candidate):
                    entity_cluster.add(candidate)
                    matches_set.add((id, candidate))
                    matches_set.add((candidate, id))
                    to_analyze.append(candidate)
                else:
                    not_matches_set.add((id, candidate))
                    not_matches_set.add((candidate, id))

        seen.update(entity_cluster)
        result.append(entity_cluster)
    return result, comparisons

In [6]:
# Run batch processing for comparison
start = time.time()

results, batch_comparisons = match_batch(candidates)

batch_elapsed_time = time.time() - start

In [7]:
# Change if your id column is different
resolved = pd.DataFrame([resolve_batch(data[data["_id"].isin(entity)]) for entity in results])
# Change to the where query you used on BrewER, and the order by
batch_filtered = resolved[resolved["brand"] == "sony"].sort_values(
    "mp", ascending=False
)
batch_filtered_pairs = set()
for group in (results[index] for index in batch_filtered.index):
    if len(group) > 1:
        batch_filtered_pairs.update([tuple(sorted(pair)) for pair in combinations(group, 2)])
    else:
        batch_filtered_pairs.add(tuple(group))

In [8]:
print("Batch filtered pairs:")
batch_filtered_pairs

Batch filtered pairs:


{('www.ebay.com//53965', 'www.walmart.com//800'),
 ('www.ebay.com//23894', 'www.ebay.com//55524'),
 ('www.ebay.com//44472', 'www.ebay.com//52649'),
 ('www.ebay.com//44798', 'www.ebay.com//47094'),
 ('www.ebay.com//43650', 'www.ebay.com//59501'),
 ('www.buzzillions.com//2821', 'www.ebay.com//44472'),
 ('www.ebay.com//56345', 'www.gosale.com//1411'),
 ('www.ebay.com//44472', 'www.gosale.com//1427'),
 ('www.ebay.com//48416', 'www.ebay.com//54824'),
 ('www.ebay.com//24382', 'www.gosale.com//1010'),
 ('www.ebay.com//56480', 'www.gosale.com//1158'),
 ('www.henrys.com//106',),
 ('www.ebay.com//46475', 'www.ebay.com//54036'),
 ('www.ebay.com//43796', 'www.gosale.com//459'),
 ('www.ebay.com//44753', 'www.ebay.com//48548'),
 ('www.ebay.com//46633', 'www.gosale.com//1158'),
 ('www.buzzillions.com//2955', 'www.ebay.com//57249'),
 ('buy.net//6510', 'www.ebay.com//59331'),
 ('www.buzzillions.com//2821', 'www.ebay.com//42565'),
 ('www.ebay.com//42559', 'www.gosale.com//759'),
 ('www.ebay.com//43636',

In [9]:
print("Batch comparisons:", batch_comparisons)
print("Batch elapsed time:", batch_elapsed_time)
print("Batch emitted:", len(batch_filtered))

Batch comparisons: 22073
Batch elapsed time: 59.14668393135071
Batch emitted: 206


In [10]:
# Prepare gold standard for BrewER matching function
gold = set()
for _, match in matches.iterrows():
    gold.add((match["l_id"], match["r_id"]))
    gold.add((match["r_id"], match["l_id"]))

In [11]:
# Initialize BrewER results tracking
brewer_start = None
brewer_results = []
brewer_entities = []

def track_brewer_results(entity):
    global brewer_start, brewer_results, brewer_entities

    brewer_entities.append(entity)
    i = len(brewer_entities) - 1

    cluster = entity.get("matches", [entity["_id"]]) 
    if isinstance(cluster, (set, list)) and len(cluster) > 1:
        cluster_pairs = set(tuple(sorted(p)) for p in combinations(cluster, 2))
    else:
        cluster_pairs = set()

    tp = len(cluster_pairs & batch_filtered_pairs)
    fp = len(cluster_pairs - batch_filtered_pairs)

    correct = (round(entity["mp"], 3) == round(batch_filtered.iloc[i]['mp'], 3)) if i < len(batch_filtered) else False


    brewer_results.append({
        "elapsed_time": entity.get("time", time.time() - brewer_start),
        "comparisons": entity.get("comparisons", 0),
        "correct": correct,
        "tp": tp,
        "fp": fp,
    })


In [12]:
# Configure BrewER task
# Create a SQL query for BrewER - must follow the exact syntax expected by the parser
# This replicates the same query from the BlendER experiments.ipynb:
# - SELECT: VOTE(_id), VOTE(description), VOTE(brand), MIN(price), AVG(mp)
# - WHERE: brand = "sony" 
# - ORDER BY: mp DESC
sql_query = '''
SELECT VOTE(_id), VOTE(description), VOTE(brand), MIN(price), AVG(mp)
FROM alaska_cameras_small
GROUP BY ENTITY WITH MATCHER bf_sparker_mf_gt
HAVING VOTE(brand) = "sony"
ORDER BY AVG(mp) DESC
'''

print("Configuring BrewER task...")

# Create task object
class SimpleSQL:
    def __init__(self, value):
        self.value = value

sql_obj = SimpleSQL(sql_query)
parsed_query = parser(sql_obj)

if parsed_query["complete"]:
    task = Task(parsed_query)
    print("Task created successfully!")
    print(f"Dataset: {task.ds}")
    print(f"Blocking function: {task.blocking_function}")
    print(f"Matching function: {task.matching_function}")
    print(f"Ordering key: {task.ordering_key}")
    print(f"Ordering mode: {task.ordering_mode}")
    print(f"Aggregation functions: {task.aggregation_functions}")
else:
    print("Error: SQL query parsing failed!")
    print("Parsed query:", parsed_query)

Configuring BrewER task...
Task created successfully!
Dataset: alaska_cameras_small
Blocking function: SparkER (Meta-Blocking)
Matching function: Ground Truth
Ordering key: mp
Ordering mode: desc
Aggregation functions: {'_id': 'vote', 'description': 'vote', 'brand': 'vote', 'price': 'min', 'mp': 'avg'}


In [13]:
# Run BrewER algorithm (reutilizando a função matcher do batch)
if 'task' in locals() and parsed_query["complete"]:
    brewer_start = time.time()
    brewer_results = []
    brewer_entities = []

    # gold fica sem uso aqui; a função matcher do batch será chamada diretamente
    gold = []  # apenas placeholder

    # Candidates
    candidates_set = set(candidates)

    try:
        import importlib, brewer as brewer_mod
        importlib.reload(brewer_mod)

        # matching do Brewer delega para o seu matcher do batch
        def _my_matching(left_id, right_id, _gold_ignored):
            return matcher(left_id, right_id)  # usa a MESMA lógica do batch
        brewer_mod.matching = _my_matching

        results_df = brewer_mod.brewer(
            task, data, gold, candidates_set,
            demo=False, mode="scratch", results=[]
        )

        print(f"BrewER completed successfully! Emitted {len(results_df)} entities.")

        for _, entity in results_df.iterrows():
            track_brewer_results(entity.to_dict())

    except Exception as e:
        print(f"Error running BrewER: {e}")
        results_df = pd.DataFrame()

else:
    print("Cannot run BrewER: task creation failed or query parsing failed.")
    results_df = pd.DataFrame()



BrewER is running: setup started.

Setup completed... let's go!

{'_id': 0,
 'brand': 'sony',
 'comparisons': 2,
 'description': 'sony a7r black interchangeable lens digital slr camera body '
                'only (36.4 mp, memory stick/sd card slot) price comparison at '
                'buy.net',
 'matches': ['www.gosale.com//1215', 'buy.net//5881'],
 'mp': np.float64(36.4),
 'price': np.float64(1586.99),
 'time': 1.2859959602355957}
{'_id': 1,
 'brand': 'sony',
 'comparisons': 2,
 'description': 'sony alpha a7r 36 4 mp full frame mirrorless camera body open '
                'box $1 027242874787 | ebay',
 'matches': ['www.ebay.com//53668'],
 'mp': np.float64(36.4),
 'price': np.float64(nan),
 'time': 1.2897229194641113}
{'_id': 2,
 'brand': 'sony',
 'comparisons': 412,
 'description': 'sony alpha ilce 6000l 24 3mp slr digital camera with e pz 16 '
                '50mm lens black | ebay',
 'matches': ['www.ebay.com//46949'],
 'mp': np.float64(24.3),
 'price': np.float64(nan),
 'tim

In [14]:
print(results_df.columns.tolist())
print(results_df[["matches", "_id", "mp"]].head())


['_id', 'description', 'brand', 'price', 'mp', 'matches', 'comparisons', 'time']
                                             matches  _id    mp
0              [www.gosale.com//1215, buy.net//5881]    0  36.4
1                              [www.ebay.com//53668]    1  36.4
2                              [www.ebay.com//46949]    2  24.3
3  [www.ebay.com//59882, www.gosale.com//531, www...    3  24.3
4                              [www.ebay.com//42367]    4  24.3


In [ ]:
# Create evaluation dataframe
brewer_dataframe = pd.DataFrame(brewer_results)

if len(brewer_dataframe) > 0:
    brewer_dataframe["running_tp"] = brewer_dataframe["tp"].cumsum()
    brewer_dataframe["running_fp"] = brewer_dataframe["fp"].cumsum()
    brewer_dataframe["running_fn"] = len(batch_filtered_pairs) - brewer_dataframe["running_tp"]
    brewer_dataframe["running_correct"] = brewer_dataframe["correct"].cumsum()

    brewer_dataframe["recall"] = (brewer_dataframe["running_tp"] / (brewer_dataframe["running_tp"] + brewer_dataframe["running_fn"])).replace(np.nan, 0)
    brewer_dataframe["precision"] = (brewer_dataframe["running_tp"] / (brewer_dataframe["running_tp"] + brewer_dataframe["running_fp"])).replace(np.nan, 0)

    brewer_dataframe["accuracy"] = brewer_dataframe["running_correct"] / len(brewer_dataframe)

    brewer_dataframe["f1"] = (
        2
        * (brewer_dataframe["precision"] * brewer_dataframe["recall"])
        / (brewer_dataframe["precision"] + brewer_dataframe["recall"])
    )

    brewer_dataframe.to_csv("output/brewer_results.csv", index=False)
else:
    print("No results from BrewER to evaluate.")

In [17]:
brewer_dataframe

,elapsed_time,comparisons,correct,tp,fp,running_tp,running_fp,running_fn,running_correct,recall,precision,accuracy,f1
0,1.285996,2,True,1,0,1,0,3507,1,0.000285,1.0,0.001776,0.000570
1,1.289723,2,False,0,0,1,0,3507,1,0.000285,1.0,0.001776,0.000570
2,2.392334,412,True,0,0,1,0,3507,2,0.000285,1.0,0.003552,0.000570
3,2.395238,412,True,10,0,11,0,3497,3,0.003136,1.0,0.005329,0.006252
4,2.397887,412,True,0,0,11,0,3497,4,0.003136,1.0,0.007105,0.006252
...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,15.167364,4614,False,0,0,241,0,3267,10,0.068700,1.0,0.017762,0.128568
559,15.170016,4614,False,0,0,241,0,3267,10,0.068700,1.0,0.017762,0.128568
560,15.172420,4614,False,0,0,241,0,3267,10,0.068700,1.0,0.017762,0.128568
561,15.175070,4614,False,0,0,241,0,3267,10,0.068700,1.0,0.017762,0.128568


In [ ]:
# Display BrewER results
print("\n=== BrewER RESULTS ===")

if len(brewer_dataframe) > 0:
    print("BrewER Performance Metrics:")
    print(brewer_dataframe)
    print(f"\nFinal Metrics:")
    print(f"- Final Precision: {brewer_dataframe['precision'].iloc[-1]:.4f}")
    print(f"- Final Recall: {brewer_dataframe['recall'].iloc[-1]:.4f}")
    print(f"- Final Accuracy: {brewer_dataframe['accuracy'].iloc[-1]:.4f}")
else:
    print("No performance metrics available from BrewER.")

if 'results_df' in locals() and len(results_df) > 0:
    print(f"\nBrewER entities emitted ({len(results_df)}):")
    print(results_df.head(10))
    results_df.to_csv("output/brewer_entities.csv", index=False)
else:
    print("No entities were emitted by BrewER.")


=== BrewER RESULTS ===
BrewER Performance Metrics:
     elapsed_time  comparisons  correct  tp  fp  running_tp  running_fp  \
0        1.285996            2     True   1   0           1           0   
1        1.289723            2    False   0   0           1           0   
2        2.392334          412     True   0   0           1           0   
3        2.395238          412     True  10   0          11           0   
4        2.397887          412     True   0   0          11           0   
..            ...          ...      ...  ..  ..         ...         ...   
558     15.167364         4614    False   0   0         241           0   
559     15.170016         4614    False   0   0         241           0   
560     15.172420         4614    False   0   0         241           0   
561     15.175070         4614    False   0   0         241           0   
562     15.178350         4614    False   0   0         241           0   

     running_fn  running_correct    recall  pre

In [23]:
# Comparison summary
print("\n" + "="*50)
print("COMPARISON SUMMARY: Batch vs BrewER")
print("="*50)

print(f"\nBatch Processing (Baseline):")
print(f"  - Comparisons: {batch_comparisons:,}")
print(f"  - Elapsed time: {batch_elapsed_time:.4f}s")
print(f"  - Entities emitted: {len(batch_filtered)}")

if len(brewer_dataframe) > 0:
    total_brewer_comparisons = brewer_dataframe["comparisons"].iloc[-1] if "comparisons" in brewer_dataframe.columns else 0
    total_brewer_time = brewer_dataframe["elapsed_time"].max() if "elapsed_time" in brewer_dataframe.columns else 0
    print(f"\nBrewER Processing (On-Demand):")
    print(f"  - Comparisons: {total_brewer_comparisons:,}")
    print(f"  - Elapsed time: {total_brewer_time:.4f}s")
    print(f"  - Entities emitted: {len(brewer_dataframe)}")
    
    if total_brewer_comparisons > 0 and batch_comparisons > 0:
        comp_reduction = ((batch_comparisons - total_brewer_comparisons) / batch_comparisons * 100)
        time_reduction = ((batch_elapsed_time - total_brewer_time) / batch_elapsed_time * 100)
        print(f"\nEfficiency Gains:")
        print(f"  - Comparison reduction: {comp_reduction:.2f}%")
        print(f"  - Time reduction: {time_reduction:.2f}%")
        print(f"  - Speedup factor: {batch_elapsed_time/total_brewer_time:.2f}x" if total_brewer_time > 0 else "")
else:
    print(f"\nBrewER Processing: No results to compare")

print("\n" + "="*50)
print("Query Configuration (same as BlendER experiments):")
print("  - SELECT: VOTE(_id), VOTE(description), VOTE(brand), MIN(price), AVG(mp)")
print("  - WHERE: brand = 'sony'")
print("  - ORDER BY: mp DESC")
print("  - LIMIT: TOP 10")
print("="*50)


COMPARISON SUMMARY: Batch vs BrewER

Batch Processing (Baseline):
  - Comparisons: 22,073
  - Elapsed time: 59.1467s
  - Entities emitted: 206

BrewER Processing (On-Demand):
  - Comparisons: 4,614
  - Elapsed time: 15.1783s
  - Entities emitted: 563

Efficiency Gains:
  - Comparison reduction: 79.10%
  - Time reduction: 74.34%
  - Speedup factor: 3.90x

Query Configuration (same as BlendER experiments):
  - SELECT: VOTE(_id), VOTE(description), VOTE(brand), MIN(price), AVG(mp)
  - WHERE: brand = 'sony'
  - ORDER BY: mp DESC
  - LIMIT: TOP 10


In [24]:
# Side-by-side comparison of results
print("\n" + "="*80)
print("SIDE-BY-SIDE RESULTS COMPARISON")
print("="*80)

print("\nBatch Processing Results (sorted by mp DESC):")
if len(batch_filtered) > 0:
    display_cols = ['_id', 'brand', 'mp', 'price'] if 'price' in batch_filtered.columns else ['_id', 'brand', 'mp']
    print(batch_filtered[display_cols].head(10).to_string(index=False))
else:
    print("No batch results to display")

print(f"\nBrewER On-Demand Results:")
if 'results_df' in locals() and len(results_df) > 0:

    available_cols = [col for col in ['_id', 'brand', 'mp', 'price', 'comparisons', 'time'] if col in results_df.columns]
    print(results_df[available_cols].head(10).to_string(index=False))
else:
    print("No BrewER results to display")

print("\n" + "="*80)


SIDE-BY-SIDE RESULTS COMPARISON

Batch Processing Results (sorted by mp DESC):
                _id brand   mp   price
      buy.net//5881  sony 36.4 1586.99
      buy.net//4330  sony 24.3     NaN
      buy.net//6531  sony 24.3 1172.99
www.ebay.com//43490  sony 24.3     NaN
www.ebay.com//42034  sony 24.3  827.99
  cammarkt.com//229  sony 24.3     NaN
www.ebay.com//42742  sony 24.3 1655.97
www.gosale.com//540  sony 24.3 1379.99
www.ebay.com//54853  sony 24.3     NaN
www.ebay.com//46677  sony 24.3     NaN

BrewER On-Demand Results:
 _id brand   mp   price  comparisons     time
   0  sony 36.4 1586.99            2 1.285996
   1  sony 36.4     NaN            2 1.289723
   2  sony 24.3     NaN          412 2.392334
   3  sony 24.3  827.99          412 2.395238
   4  sony 24.3     NaN          412 2.397887
   5  sony 24.3     NaN          412 2.400558
   6  sony 24.3     NaN          412 2.403246
   7  sony 24.3     NaN          412 2.406068
   8  sony 24.3     NaN          412 2.408704
   9

In [25]:
from itertools import combinations

def pairs_of(ids):
    return {tuple(sorted(p)) for p in combinations(ids, 2)}

pred_pairs = set()
if "matches" in results_df.columns:
    for m in results_df["matches"]:
        pred_pairs |= pairs_of(m)

print("pares preditos:", len(pred_pairs))
print("pares baseline:", len(batch_filtered_pairs))
print("TP:", len(pred_pairs & batch_filtered_pairs))
print("FP:", len(pred_pairs - batch_filtered_pairs))  # idealmente 0
print("FN:", len(batch_filtered_pairs - pred_pairs))  # se >0, é blocking/HAVING


pares preditos: 241
pares baseline: 3508
TP: 241
FP: 0
FN: 3267
